### Lab 2: Hyponyms and Hypernyms

In [0]:
import pandas as pd
import wikipedia
import os
from tqdm import tqdm
from joblib import Parallel, delayed
import multiprocessing
import networkx as nx
import matplotlib.pyplot as plt
from collections import defaultdict
from ipywidgets import interact, interactive, fixed, interact_manual,widgets
from IPython.display import display
import json
num_cores = multiprocessing.cpu_count()
print(num_cores)
wikipedia.set_lang("ru")
# DATA_PATH_LIST = ['D:','src2','taxonomy-enrichment','data','training_data']
# DATA_PATH_LIST = ['/content']
# EMBEDDING_MODEL_FILENAME = "wiki_node2vec.bin"
# DATA_PATH="/".join(DATA_PATH_LIST+["training_nouns.tsv"])
# df = pd.read_csv(DATA_PATH,sep='\t')
%matplotlib inline

4


In [0]:
from google.colab import drive
drive.mount('/content/drive')

In [0]:
pip install git+https://github.com/lucasdnd/Wikipedia.git

  Cloning https://github.com/lucasdnd/Wikipedia.git to /tmp/pip-req-build-cj6gkxug
  Running command git clone -q https://github.com/lucasdnd/Wikipedia.git /tmp/pip-req-build-cj6gkxug
  Created wheel for wikipedia: filename=wikipedia-1.4.0-cp36-none-any.whl size=11696 sha256=7a6725c88b777458e3ac42806e37c302e80df77a629814a34211621604ef23b6
  Stored in directory: /tmp/pip-ephem-wheel-cache-vre9fi8z/wheels/f9/8b/8f/53af797c01eebc4990d7dafdd35399df459c641556cff9ec44
Successfully built wikipedia


In [0]:
def prestr(x):
    return str(x).replace('\"','').replace("'",'"')

In [0]:
class DefDict(defaultdict):
    def __missing__(self, key):
        self[key] = key
        return key
    
idx2syns = DefDict(lambda x:x)
for val in df.values:
    idx2syns[val[0]]=val[1]
    try:
        pidxs = json.loads(prestr(val[2]))
        concp = [el.split(",")[0] for el in json.loads(prestr(val[3]))]
        idx2syns.update(dict(zip(pidxs,concp)))
    except:
        print(prestr(val[2]))
        print(prestr(val[3]))

["3240-N", "145191-N"]
["СП


### Interactive visualization of hyponyms and hypernyms

In [0]:
button = widgets.Button(description="Draw")
query = widgets.Text(
    value='МАТЬ',
    placeholder='Query',
    description='String:',
    disabled=False
)
display(button,query)




def creategraph(df):
    res = []
    for row in df.values:
        cohyps = row[1].split(",")
        for idx,cohyp in enumerate(cohyps):
            for parent in json.loads(prestr(row[2])):
                res.append((row[0]+'-'+str(idx),parent))
    return res

def graphdraw(b):
    print("graphdraw",query.value)
    subset = df[df['TEXT'].str.contains(query.value.upper())]
    g = nx.DiGraph()
    for el in subset.values:
        cohyps = el[1].split(",")
        print(cohyps)
        syns = idx2syns[el[0]]
        for child in cohyps:
            for parent in json.loads(prestr(el[2])):
                ed = g.add_edge(child,idx2syns[parent],label="is a")
            
    plt.figure(figsize=(15,15))
    pos = nx.nx_agraph.graphviz_layout(g)
    nx.draw(g,with_labels=True,pos=pos)
#     edge_labels=nx.draw_networkx_edge_labels(g,pos=pos)
    plt.show()
button.on_click(graphdraw)

Button(description='Draw', style=ButtonStyle())

Text(value='МАТЬ', description='String:', placeholder='Query')

In [0]:
!pip install requests
!pip install backoff
!apt-get install python-pygraphviz
!apt-get install graphviz-dev
!pip install graphviz
!pip install eralchemy
!pip install git+https://github.com/innovationOUtside/ipython_magic_eralchemy.git


Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following package was automatically installed and is no longer required:
  libnvidia-common-430
Use 'apt autoremove' to remove it.
Suggested packages:
  python-pygraphviz-doc
The following NEW packages will be installed:
  python-pygraphviz
0 upgraded, 1 newly installed, 0 to remove and 25 not upgraded.
Need to get 71.6 kB of archives.
After this operation, 410 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic-updates/universe amd64 python-pygraphviz amd64 1.4~rc1-1build2.1 [71.6 kB]
Fetched 71.6 kB in 1s (112 kB/s)
Selecting previously unselected package python-pygraphviz.
(Reading database ... 134443 files and directories currently installed.)
Preparing to unpack .../python-pygraphviz_1.4~rc1-1build2.1_amd64.deb ...
Unpacking python-pygraphviz (1.4~rc1-1build2.1) ...
Setting up python-pygraphviz (1.4~rc1-1build2.1) ...
Reading package lists... Done

### Pattern extractor

Yargy — библиотека для извлечения структурированной информации из текстов на русском языке. Правила описываются контекстно-свободными грамматиками и словарями ключевых слов. Банк готовых правил для имён, дат, адресов и других сущностей доступен в репозитории Natasha.
* https://yargy.readthedocs.io/ru/latest/
* http://pymorphy2.readthedocs.io/en/latest/user/grammemes.html
* https://github.com/natasha/natasha

### Токенизатор

In [0]:
from yargy.tokenizer import MorphTokenizer


tokenizer = MorphTokenizer()
text = '''Ростов-на-Дону
Длительностью 18ч. 10мин.
Яндекс.Такси
π ≈ 3.1415
1 500 000$
http://vk.com
'''
for line in text.splitlines():
    print([_.value for _ in tokenizer(line)])

['Ростов', '-', 'на', '-', 'Дону']
['Длительностью', '18', 'ч', '.', '10', 'мин', '.']
['Яндекс', '.', 'Такси']
['π', '≈', '3', '.', '1415']
['1', '500', '000', '$']
['http', ':', '/', '/', 'vk', '.', 'com']


In [0]:
%pip install yargy

     |████████████████████████████████| 51kB 2.7MB/s 
     |████████████████████████████████| 51kB 4.4MB/s 
     |████████████████████████████████| 7.1MB 7.4MB/s 


# Газеттир
Газеттир нужен для удобной работы с последовательностью слов. Например, можно написать:

In [0]:
from yargy import or_, rule
from yargy.predicates import normalized

RULE = or_(
    rule(normalized('dvd'), '-', normalized('диск')),
    rule(normalized('видео'), normalized('файл'))
)

In [0]:
from yargy import Parser
from yargy.pipelines import morph_pipeline


RULE = morph_pipeline([
    'dvd-диск',
    'видео файл',
    'видеофильм',
    'газета',
    'электронный дневник',
    'эссе',
])

parser = Parser(RULE)
text = 'Видео файл на dvd-диске'
for match in parser.findall(text):
    print([_.value for _ in match.tokens])

['Видео', 'файл']
['dvd', '-', 'диске']


In [0]:
from yargy import Parser, rule, and_
from yargy.predicates import gram, is_capitalized, dictionary


GEO = rule(
    and_(
        gram('ADJF'),  # так помечается прилагательное, остальные пометки описаны в
                       # http://pymorphy2.readthedocs.io/en/latest/user/grammemes.html
        is_capitalized()
    ),
    gram('ADJF').optional().repeatable(),
    dictionary({
        'федерация',
        'республика'
    })
)


parser = Parser(GEO)
text = '''
В Чеченской республике на день рождения ...
Донецкая народная республика провозгласила ...
Башня Федерация — одна из самых высоких ...
'''
for match in parser.findall(text):
    print([_.value for _ in match.tokens])

['Чеченской', 'республике']
['Донецкая', 'народная', 'республика']


### Предикаты

Предикат — функция, которая принимает на вход токен и возвращает True или False. В Yargy встроено много готовых предикатов. Полный список есть в справочнике. Предикаты комбинируются с помощью and_, or_ и not_.

In [0]:
from yargy import and_, not_
from yargy.tokenizer import MorphTokenizer
from yargy.predicates import is_capitalized, eq


tokenizer = MorphTokenizer()
token = next(tokenizer('Стали'))

predicate = is_capitalized()
print(predicate(token))

predicate = and_(
    is_capitalized(),
    not_(eq('марки'))
)
print(predicate(token))

True
True


### Грамматики
В Yargy используется специальный DSL для описания грамматик. Любую контекстно-свободную грамматику можно описать с помощью конструкций Питона. Например, есть примитивная грамматика для размеров одежды:

KEY -> р. | размер

VALUE -> S | M | L

SIZE -> KEY VALUE



In [0]:
from yargy import rule, or_


KEY = or_(
    rule('р', '.'),
    rule('размер')
).named('KEY')
VALUE = or_(
    rule('S'),
    rule('M'),
    rule('L'),
    rule('XS'),
).named('VALUE')
SIZE = rule(
    KEY,
    VALUE
).named('SIZE')
SIZE.normalized.as_bnf

SIZE -> KEY VALUE
KEY -> 'р' '.' | 'размер'
VALUE -> 'S' | 'M' | 'L' | 'XS'


In [0]:
parser = Parser(
    SIZE
)
text = 'размер M; размер A; размер XS;'
for match in parser.findall(text):
    print([_.value for _ in match.tokens])

['размер', 'M']
['размер', 'XS']


In [0]:
from yargy import Parser, rule, and_, or_, not_
from yargy.interpretation import fact, attribute
from yargy.predicates import gram, is_capitalized, dictionary, eq
import re
import pandas as pd
from tqdm import tqdm_notebook
from gensim import utils
from yargy.morph import MorphAnalyzer as ma
import pymorphy2 as ppp

In [0]:
ADJP = or_(
    rule(
        ADJF.optional().repeatable(1,3),
        eq(',').optional(),
        gram('PREP').optional().repeatable(1,3),
        eq(',').optional(),
        gram('PRTF').optional().repeatable(1,3)
    ),
    rule(
        gram('ADJF').repeatable(1,3).optional(),
        gram('PREP').optional(),
        gram('ADJF').repeatable(1,3).optional()
    ),
    rule(
        gram('PREP').optional(),
        gram('PRTF').repeatable(1,3).optional(),
        gram('PREP').optional()
    ),
    rule(
        gram('ADJF').repeatable(1,3).optional(),
        gram('PRTF').repeatable(1,3).optional(),
        gram('ADJF').repeatable(1,3).optional()
    )

)
NF = rule(
    gram('ADJF').optional().repeatable(1,3),
    gram('NOUN').optional().repeatable(1,3),
    or_(
              eq('или').optional(),
              eq('и').optional(),
              eq(',').optional()
          )
).repeatable(1, 3)

ADJF = or_(
        rule(
          or_(
              gram('ADJF').optional(),
              gram('ADVB').optional()
          ),
          eq('-'),
          gram('ADJF')
        ),
      rule(gram('ADJF')
      )
)
NOUN = or_(
        rule(
          or_(
              gram('NOUN').optional(),
              gram('ADVB').optional()
          ),
          eq('-'),
          gram('NOUN')
        ),
        rule(gram('NOUN')
        )
)

     

START = rule(
    or_(
        rule(gram('ADJF')),
        #ADJP,
        rule(gram('NOUN'))
    ).optional(),
    gram('NOUN')
)

START_S = or_(
    eq('такой'),
    eq('такие'),
)

KAK = eq('как')
INCLUDING = or_(
    or_(
        eq('в'),
        eq('том'),
        eq('числе'),
    ),
    eq('включающий'),
    or_(
        eq('включающий'),
        eq('в'),
        eq('себя'),
    ),
    or_(
        eq('включающие'),
        eq('в'),
        eq('себя'),
    ),
    eq('включающие'),
    eq('особенно'),

)

MID_S = or_(
    rule(
        or_(
            eq('такой'),
            eq('такие'),
        ),
        eq('как')
    ),
    rule(
        or_(
            eq('.').optional(),
            eq(',').optional()
        ),
        eq('к'),
        or_(
            eq('ним'),
            eq('нему'),
            eq('ней')
        ),
        or_(
            eq('относятся'),
            eq('относится'),
            eq('относят ')
        )
    )
)
ATAKJE = rule(
    eq(','),
    eq('а'),
    eq('также')
)

ZNACHENIE = rule(
    eq('значение'),
    or_(
      eq('-'),
      eq(':')  
    )
)
UTOCH = rule(
    #eq(', '),
    eq('или'),
    #eq('элитры ')
    gram('NOUN')#.repeatable(1,3)
    #ADJP
    
)
PEREVOD = rule(
    eq('('),
    #gram('NOUN').repeatable(),
    eq('лат.'),
    gram('NOUN').repeatable(1,3),
    eq(')')
)
MID = or_(
    rule(
        eq('это')
    ),
    rule(
        or_(
              eq('—'),
              eq('–')
        )
    ),
    rule(
        or_(
              eq('—'),
              eq('–')
        ),
        eq('это')
    ),
    rule(
        or_(
              eq('—'),
            eq('–')
        ),
        not_(eq('км'))
    ),
    rule(
        or_(
            eq('и'),
            eq('или')
        ),
        eq('другие')
    ),
    rule(
        eq('представляет'),
        or_(
            eq('собой'),
            eq('из себя')
        )
    )
)

END = or_(
    rule(
        gram('NOUN'),
        gram('NOUN')
    ),
    rule(
        ADJF.repeatable(),
        #ADJP.optional(),
        NOUN.repeatable()
    ),
    rule(
        gram('ADJF'),
        rule(
          or_(
              eq('или'),
              eq('и')
          )
        ),
        gram('NOUN').repeatable(1,3),
        gram('ADJF').repeatable(1,3),
        gram('NOUN').repeatable(1,3),
    ),
    rule(
        gram('NOUN').repeatable(),
        gram('ADJF'),
        gram('NOUN').repeatable()
    ),
    rule(
        gram('ADJF'),
        eq(',') ,
        gram('ADJF'),
        gram('NOUN').repeatable()
    ),
    rule(
        NOUN.optional().repeatable(1, 3),
        ADJP.optional(),
        gram('VERB').optional(),
        gram('NOUN').repeatable(1, 3).optional()
    )
    # rule(
    #     eq('.').optional(),
    #     MID_S,
    #     NF.repeatable(1,3) 
    # )
)

Item = fact(
    'Item',
    [attribute('titles').repeatable()]
)


IGNORE = rule(
    '(',
    not_(eq(')')).repeatable(),
    ')'
)

ITEM = rule(
    IGNORE.interpretation(
        Item.titles
    ),
    eq(',').optional() 
).repeatable().interpretation(
    Item
)

In [0]:
def get_hyperonyms(main_word):
    HYPONYM = eq(utils.deaccent(main_word))
    RULE = or_(
        rule(HYPONYM, ATAKJE, START, MID, END),
        rule(HYPONYM, MID, END),
        rule(START_S, END, KAK, HYPONYM),
        rule(END, INCLUDING, HYPONYM)
    )

    parser = Parser(RULE) 
    if(wikipedia.suggest(main_word) == None):
      text = utils.deaccent(wikipedia.summary(main_word))
      #print("test true")
    else:
      #print("test falsee")
      text = utils.deaccent(wikipedia.summary(main_word,sentences=2, auto_suggest=False))
    print(text)
    #text = re.sub(r'\(.+?\)', '', text)
    text = text.lower().replace('* сергии радонежскии* ', '')

    hyperonyms = []
    for idx, match in enumerate(parser.findall(text.lower())):
        k = [_.value for _ in match.tokens]
        hyperonyms.append(k)
        print(k)
        

    return hyperonyms

In [0]:
ds = pd.read_excel('/content/hyponym-hypernym.xls')
#print(ds.head(5))
words = ds['гипоним'].unique().tolist()
print(len(words))
final_hyperonyms = []
for i in range(len(words)):
  #print(type(words[i]))
  words[i] = words[i].lower()
  print("Hyperonym for "+ str(words[i]) + " :" )
  final_hyperonyms.append(get_hyperonyms(words[i]))

print("**************************\n")
print("Гипонимы - гиперонимы:\n")
print(*final_hyperonyms, sep = "\n")
  



In [0]:
def get_hyperonyms_fromDF(main_word, text_source):
    HYPONYM = eq(utils.deaccent(main_word))
    RULE = or_(
        rule(HYPONYM, ATAKJE, START, MID, END),
        rule(HYPONYM, MID, END),
        rule(START_S, END, KAK, HYPONYM),
        rule(END, INCLUDING, HYPONYM),
        rule(HYPONYM, ZNACHENIE, MID, END),
       # rule(HYPONYM, PEREVOD, MID, END)
        rule(HYPONYM, UTOCH, MID, END),
        rule( START, MID_S, NF, HYPONYM)
        #rule(HYPONYM, UTOCH, PEREVOD, MID, END)
        
    )

    parser = Parser(RULE) 

    accents = []
    for i in range(len(text_source)):
      accents.append(text_source[i]=='й')
    text = []
    #text = utils.deaccent(text_source) # litter deaccent
    pop = ['́', '́ ']
    for i in range(len(text_source)):
      if text_source[i] not in pop :
        text.append(text_source[i])  # litter deaccent
    
    text = "".join(text) 
    # #tmp_str = "".join(accents)
    # excluse = ['́', '́']
    # for j in range(len(text)): # remiving accent  
    #   if( text_source[j] not in text and (text_source[j] != 'й' and text_source[j] != 'ё')):
    #       print(str(text_source[j])+" ")
    #       accents.pop(j)
    # if len(text)!=len(accents):
    #     print("test failed" + str(len(text))+" " + str(len(accents)))

    # for i in range(len(text)):
    #   if( accents[i]==True ):
    #     #text_source = text_source[:i] + 'й' + text_source[(i+1):]
    #     #text_source.replacce(text_source[i], 'й')
    #     text = list(text)
    #     text[i] = 'й'
    #     text = "".join(text) 

    print(text)
    text = re.sub(r'\(.+?\)', '', text)
    text = text.lower().replace('* сергии радонежскии* ', '')

    hyperonyms = []
    for idx, match in enumerate(parser.findall(text.lower())):
        k = [_.value for _ in match.tokens]
        hyperonyms.append(k)
        print(k)
        

    return hyperonyms

In [338]:
ds = pd.read_excel('/content/hyponym-hypernym_new.xls')
#print(ds.head(5))
gyponyms = ds['гипоним'].tolist()
hyperonyms_fromdf = ds['гипероним'].tolist() 
text_fromdf = ds['пример'].tolist()
hypo_hyper_map = {}

for word in ds['гипоним'].unique().tolist():
  hypo_hyper_map[word.lower()] = []

print(hypo_hyper_map)

#print(len(words))
final_hyperonyms = []
if len(gyponyms)!=len(text_fromdf):
  print("Количество гипонимов и примеров не равно. ")
  exit

cnt = 0
for i in range(len(gyponyms)):
  #print(type(words[i]))
  gyponyms[i] = gyponyms[i].lower()
  #print("Hyperonym for "+ str(words[i]) + " :" )
  hyperonym_i = get_hyperonyms_fromDF(gyponyms[i], text_fromdf[i])
  print(hypo_hyper_map[gyponyms[i]])
  if hyperonym_i not in hypo_hyper_map[gyponyms[i]]:
    hypo_hyper_map[gyponyms[i]].append(hyperonym_i)


# for word_i in range(len(hypo_hyper_map)):
#   #print(type(words[i]))
#   #gyponyms[i] = gyponyms[i].lower()
#   #print("Hyperonym for "+ str(words[i]) + " :" )
#   tmp = word_i
#   for i in range(len(tmp)):

#   hyperonym_i = get_hyperonyms_fromDF(gyponyms[i], text_fromdf[i])
#   print(hypo_hyper_map[gyponyms[i]])
#   if hyperonym_i not in hypo_hyper_map[gyponyms[i]]:
#     hypo_hyper_map[gyponyms[i]].append(hyperonym_i)
                                              
#print("**************************\n")


{'надкостница': [], 'надкрылья': [], 'наигрыш': [], 'наколенник': [], 'наличник': [], 'намагниченность': [], 'наносекунда': [], 'наноструктура': [], 'напильник': [], 'наркотизация': [], 'нарукавник': [], 'наседка': [], 'наследодатель': [], 'насмешник': [], 'настурция': [], 'натуропат': [], 'научпоп': [], 'нацбол': [], 'наяда': [], 'неврастения': [], 'невропатия': [], 'невротик': [], 'недовыполнение': [], 'недонесение': []}
Надкостница (периост) – ткань, окружающая кость снаружи.
['надкостница', '–', 'ткань', ',', 'окружающая', 'кость']
[]
надкостница – оболочка кости, состоящая из плотной волокнистой соединительной ткани. (Словарь медицинских терминов)
['надкостница', '–', 'оболочка', 'кости', ',', 'состоящая', 'из']
[[['надкостница', '–', 'ткань', ',', 'окружающая', 'кость']]]
Надкостница (периост) — соединительнотканная пленка, окружающая кость снаружи. 
['надкостница', '—', 'соединительнотканная', 'пленка', ',', 'окружающая', 'кость']
[[['надкостница', '–', 'ткань', ',', 'окружающая

In [339]:
print("Гипонимы - гиперонимы:\n")
cnt = 0
for word in hypo_hyper_map:
  cnt += 1+ len(hypo_hyper_map[word])
  print(word.upper() + '-' + str(hypo_hyper_map[word]))

print("Всего гиперонимов - " + str(cnt))  

Гипонимы - гиперонимы:

НАДКОСТНИЦА-[[['надкостница', '–', 'ткань', ',', 'окружающая', 'кость']], [['надкостница', '–', 'оболочка', 'кости', ',', 'состоящая', 'из']], [['надкостница', '—', 'соединительнотканная', 'пленка', ',', 'окружающая', 'кость']], [], [['надкостница', 'представляет', 'собой', 'плотную', 'соединительнотканную', 'пластинку']]]
НАДКРЫЛЬЯ-[[]]
НАИГРЫШ-[[['наигрыш', '–', 'народная', 'инструментальная', 'мелодия']], [['наигрыш', '–', 'наигрываемая', 'мелодия', ',', 'мотив']], []]
НАКОЛЕННИК-[[['наколенник', '—', 'элемент', 'доспеха', ',', 'предназначенный']], [['наколенник', '–', 'повязка', ',', 'накладка']], [['наколенник', '–', 'это', 'спортивный', 'и']], [['наколенник', '–', 'накладка', ',', 'повязка', 'и', 'т']], []]
НАЛИЧНИК-[[['наличник', '–', 'небольшой', 'выступ', ',']], [], [['наличник', '–', 'традиционный', 'элемент', 'декора', 'фасада']], [['наличник', '—', 'деревянное']]]
НАМАГНИЧЕННОСТЬ-[[['намагниченность', '—', 'это', 'термин', ',', 'используемый']], [['н

In [0]:
s0 = "иога - хороший спорт"
s = list(s0)
print(s)
#['H', 'e', 'l', 'l', 'o', ' ', 'z', 'o', 'r', 'l', 'd']
s[0] = 'й'
s
#['H', 'e', 'l', 'l', 'o', ' ', 'W', 'o', 'r', 'l', 'd']
s = "".join(s)
print(s)

['и', 'о', 'г', 'а', ' ', '-', ' ', 'х', 'о', 'р', 'о', 'ш', 'и', 'й', ' ', 'с', 'п', 'о', 'р', 'т']
йога - хороший спорт


#### Task 1 (deadline 19.02.2020 24:00)
* Find your name on the spreadsheet https://docs.google.com/spreadsheets/d/1RR2I6toCkebbGU1UK83HS70Ru_l0_o-nnZIHyiFB0No/edit?usp=sharing. In opposite of your name there are 24 words of hyponyms, you have to insert five corresponding hypernyms next to them. Examples of hyponyms and hyperonyms relationship you can find above in the current Jupiter notebook.
* Find for each pair of hyponyms and hypernyms a corresponding snippet of a text with their mentions. The source of the text can be any free resources, e.g., Wikipedia, Google, Yandex, others. You should save the snippets and their URLs within the lab2 folder in your NLP git-repo with .csv file-extension in a single file.

#### Task 2 (deadline 26.02.2020 24:00)
* It would be best if you created a pandas DataFrame of the texts from the previous task. And apply to the DataFrame the function 'get_hyperonyms,' which must return the list of the corresponding hypernyms from the text automatically. If there are errors or misses, you should fix them in the code for your case of the 24 words. Nevertheless, it is strictly prohibited to use hard coding. Save your notebook with parser code within the lab2 folder in your NLP git-repo.